### Initialization

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

import sys
sys.path.append('../common')
from utils import process_group

fpath1 ='/home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ'
fpath2 ='/home/8ididata/2022-1/babnigg202203/cluster_results_QZ'


### Averaging of SA-XPCS results, temperature ramp up

In [2]:

avg_ramp_up, t_el, ql_dyn, ql_sta = process_group(group='E0110', 
                                              prefix=fpath2,
                                              zone_idx='auto',
                                              num_cores=24)


2023-12-20 16:25:12,802 utils                   : total number of files in E0110  is 1200
2023-12-20 16:25:13,495 utils                   : Failed to get temperature for file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0110_S3_4_Ramp_340C10p_att00_Rq0_00493_0001-100000.hdf, return nan
2023-12-20 16:25:13,653 utils                   : Failed to get temperature for file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0110_S3_4_Ramp_340C10p_att00_Rq0_00618_0001-100000.hdf, return nan
2023-12-20 16:25:14,849 xpcs_viewer.fileIO.hdf_reader: key not found: /exchange/partition-mean-total
2023-12-20 16:25:14,856 utils                   : failed to read file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0110_S3_4_Ramp_340C10p_att00_Rq0_00493_0001-100000.hdf, skip this file
2023-12-20 16:25:14,931 xpcs_viewer.fileIO.hdf_reader: key not found: /exchange/partition-mean-total
2023-12-20 16:25:14,937 utils                   : failed to read file /home/8ididata/2022-1/bab

### Averaging of SA-XPCS results, temperature ramp up

In [3]:

avg_ramp_down, _, _, _ = process_group(group='E0111', 
                                 prefix=fpath2,
                                 zone_idx='auto',
                                 num_cores=24)


2023-12-20 16:25:18,166 utils                   : total number of files in E0111  is 1200
2023-12-20 16:25:20,006 xpcs_viewer.fileIO.hdf_reader: key not found: /exchange/partition-mean-total
2023-12-20 16:25:20,013 utils                   : failed to read file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0111_S3_4_Ramp_200C10p_att00_Rq0_00485_0001-100000.hdf, skip this file
2023-12-20 16:25:20,023 xpcs_viewer.fileIO.hdf_reader: key not found: /exchange/partition-mean-total
2023-12-20 16:25:20,029 utils                   : failed to read file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0111_S3_4_Ramp_200C10p_att00_Rq0_00006_0001-100000.hdf, skip this file
2023-12-20 16:25:20,720 xpcs_viewer.fileIO.hdf_reader: key not found: /exchange/partition-mean-total
2023-12-20 16:25:20,726 utils                   : failed to read file /home/8ididata/2022-1/babnigg202203/cluster_results_QZ/E0111_S3_4_Ramp_200C10p_att00_Rq0_00777_0001-100000.hdf, skip this file
2023-12-20 16:25

### Averaging of background

In [4]:

avg_bg, _, _, _ = process_group(group='D0138',
                                 num_sections=1,
                                 prefix=fpath2,
                                 zone_idx='auto',
                                 num_cores=24)


2023-12-20 16:25:23,611 utils                   : total number of files in D0138  is 50
2023-12-20 16:25:24,791 utils                   : label='D0138_section_00': remove 7 datasets out of 50


### Data reduction

Subtracting I(Q) with background, converting to absolute scattering cross-section, and dump to pickle file for plotting

In [6]:

fname = 'E0110_S3_4_Ramp_340C10p_att00_Rq0_00001_0001-100000.hdf'

with h5py.File(fpath2+'/'+fname, 'r') as f:
    _ = f['measurement']['sample']['absolute_cross_section_scale'][()]
    abs_coeff = _[0][0]
    
for ii in range(len(avg_ramp_up)):
    avg_ramp_up[ii]["saxs_1d"] = abs_coeff*(avg_ramp_up[ii]["saxs_1d"] - avg_bg[0]["saxs_1d"])

for ii in range(len(avg_ramp_down)):
    avg_ramp_down[ii]["saxs_1d"] = abs_coeff*(avg_ramp_down[ii]["saxs_1d"] - avg_bg[0]["saxs_1d"])

with open('checkpoint.pickle', 'wb') as f:  # 'wb' for binary write mode
    pickle.dump(avg_ramp_up, f)  # Dump the object to the file
    pickle.dump(avg_ramp_down, f)
    pickle.dump(t_el, f)
    pickle.dump(ql_sta, f)
    pickle.dump(ql_dyn, f)
    